# Antes de iniciar...

- En la carpeta salidas_modelo deben estar los archivos resultantes de la ejecución anterior, donde el nombre del archivo tiene el id de la sede. Ej: 2161.xlsx

- El archivo de tablas de WFS se debe llamar "TablasWFS.xlsx"

- El archivo de profesionales se debe llamar "2_PROFESIONALES.xlsx"

- La tabla de homologación se debe llamar "stations.csv" y debe estar separado por ";"

# Instalar

In [0]:
!pip install openpyxl pysftp

---
# Importar librerias necesarias
---

In [0]:
import pandas as pd
import numpy as np
import glob
import shutil
import pysftp
import os
from datetime import datetime

---
# Definir variables comunes
---

In [0]:
lista_dias = ['DOMINGO', 'LUNES', 'MARTES', 'MIERCOLES', 'JUEVES', 'VIERNES', 'SABADO', 'FESTIVO']
lista_dias_wfs = ['Sun', 'Mon', 'Tues', 'Wed', 'Thu', 'Fri', 'Sat', 'Hol']

ruta = '/dbfs/FileStore/tables/'
ruta_entradas = ruta + 'entradas/'
ruta_salidas_modelo = ruta + 'salidas_modelo/'
ruta_salidas = ruta + 'salidas/'

ruta_salidas_sftp = '/FTPCuadrodeturnos/Modelo_predictivo/Salida/'

---
# Leer los archivos
---

## Resultado del modelo

### Leer

In [0]:
# se leen todos los archivos de la carpeta
archivos_xlsx = glob.glob(ruta_salidas_modelo + '*.xlsx')

l_dataframes = []
for archivo in archivos_xlsx:
    df_tempi = pd.read_excel(archivo, 'Hoja1') # Hoja1 que tiene el resultado
    nombre_archivo = archivo.split('/')[-1].split('_')[-1].split('.')[0] # porque el nombre del archivo es el id de la sede
    df_tempi['id_sede'] = nombre_archivo
    l_dataframes.append(df_tempi)

# Combinar los DataFrames en un solo DataFrame utilizando pd.concat()
df_modelo = pd.concat(l_dataframes, ignore_index = True)

df_modelo['persona'] = df_modelo['persona'].astype(str).str.strip() # porque si ya era un string podria tener espacios
df_modelo


### Convertir el turno en hora ini y hora fin

In [0]:
# Convertir el turno en hora ini y hora fin

# se divide
df_modelo[['num_horas', 'hora_ini']] = df_modelo['turno'].str.split('_', expand = True)

# reemplazar la t
df_modelo['num_horas'] = df_modelo['num_horas'].str.replace('T', '').astype(int)

# arreglar tipos de dato
df_modelo['hora_ini'] = df_modelo['hora_ini'].astype(int)
df_modelo['hora_fin'] = df_modelo['hora_ini'] + df_modelo['num_horas']

# ya no lo necesitamos mas
df_modelo = df_modelo.drop('turno', axis = 1)

df_modelo

## Maestro de profesionales

In [0]:
df_profesionales = pd.read_excel(ruta_entradas + '2_PROFESIONALES.xlsx', 'MAESTRA',
                         usecols = ['Id_Sede', 'Id_Compañia', 'Codigo Sf (Id Profesional)', 'QualificationCargo(IdQualification)', 'Station Name '])


df_profesionales = df_profesionales.rename(
    columns = {
        'Id_Sede' : 'id_sede',
        'Id_Compañia' : 'id_tipo_sede',
        'Codigo Sf (Id Profesional)' : 'id_persona',
        'QualificationCargo(IdQualification)' : 'cualificacion',
        'Station Name ' : 'estacion'
    }
)

df_profesionales['id_sede'] = df_profesionales['id_sede'].astype(str).str.strip() # porque si ya era un string podria tener espacios
df_profesionales['id_persona'] = df_profesionales['id_persona'].astype(str).str.strip() # porque si ya era un string podria tener espacios

df_profesionales



## Tabla homologación
Station / SchedulingGroup / SchedulingUnit

In [0]:
df_homologacion = pd.read_csv(ruta_entradas + 'stations.csv', delimiter = ';',
                              usecols = ['StationName', 'SchedulingGroup', 'SchedulingUnit'])
df_homologacion

## Tablas de validacion (WFS)

In [0]:
df_wfs = pd.read_excel(ruta_entradas + 'TablasWFS.xlsx', header = 1,
                         usecols = ['Station Name (Caracteres 20)', 'Shift Name', 'QualificationName', 'SchedulingUnit'])

v_estaciones = df_wfs['Station Name (Caracteres 20)'].dropna().drop_duplicates()
v_turnos = df_wfs['Shift Name'].dropna().drop_duplicates()
v_cualificaciones = df_wfs[['QualificationName', 'SchedulingUnit']].drop_duplicates()

v_cualificaciones



---
# Agrupar
---

## (x) Agrupar cuando el horario es continuo

In [0]:
# (Pendiente eliminar) Agrupar cuando el horario es continuo (teniendo en cuenta: persona y el dia, ignorar el consultorio)


## Obtener la hora maxima y minima para cada persona y dia

In [0]:
# (Nuevo) Obtener la hora maxima y minima para cada persona y dia
# No importa si el horario es continuo o no, igual se entrega la franja completa

df_modelo_gp = df_modelo.groupby(['id_sede', 'persona', 'dia']).agg({'hora_ini' : 'min', 'hora_fin' : 'max'})
df_modelo_gp = df_modelo_gp.reset_index()

df_modelo_gp



---
# Completar la información
---
Se debe hacer en este punto para poder construir los archivos de patrones de rotación

## Cruzar con el maestro de profesionales

In [0]:
# Traer los campos para formar el archivo plano: con la persona traer el station y qualification del maestro #2 (profesionales)

df_modelo_detallado = pd.merge(df_modelo_gp, df_profesionales, how = 'left', left_on=['id_sede', 'persona'], right_on=['id_sede', 'id_persona'])
df_modelo_detallado = df_modelo_detallado.drop('id_persona', axis = 1)
df_modelo_detallado


## Construir shift name

In [0]:
# Construir el shift name: con id_compañia poner el prefijo (22: IPS ; 28: Ay_Dx) + hora ini y hora fin

# shift name

def asignar_prefijo_turno(valor):
    if valor == 22:
        return 'IPS'
    elif valor == 28:
        return 'Ay_Dx'
    else:
        return '???'

# primero asignamos el prefijo
df_modelo_detallado['turno'] = df_modelo_detallado['id_tipo_sede'].apply(asignar_prefijo_turno)
df_modelo_detallado = df_modelo_detallado.drop('id_tipo_sede', axis = 1) # al tener el prefijo ya no lo necesitamos
df_modelo_detallado



In [0]:
df_modelo_detallado['turno'] = df_modelo_detallado['turno'] + '_' + \
df_modelo_detallado['hora_ini'].astype(str).str.zfill(2) + '00_' + \
df_modelo_detallado['hora_fin'].astype(str).str.zfill(2) + '00'

df_modelo_detallado



## Construir model name

In [0]:
# Agregar un nombre que haga unico cada registro

df_modelo_detallado['nombre_modelo'] = df_modelo_detallado['estacion'] + '_' + df_modelo_detallado['cualificacion'] + '_' + df_modelo_detallado['turno']
df_modelo_detallado


---
# Construir dfs de patrones de rotación : df_maestro_patrones y df_detalle_patrones
---

## Detalle rotación
Pasar los dias (texto) al numero que representa
1: Lunes - 2: Martes - ... - 7: Domingo - 8: Lunes 2 - 9: Martes 2 - ...

In [0]:
# Todo menos el ultimo elemento: "festivo"
lista_dias_full = lista_dias[:-1]

# Para que el domingo quede al final:
tempi = lista_dias_full.pop(0)
lista_dias_full.append(tempi)

# Agregar la lista otra vez pero con el "2"
lista_dias_full = lista_dias_full + [dia + " 2" for dia in lista_dias_full]

lista_dias_full



In [0]:
# Pasar dia a numero
df_detalle_patrones = df_modelo_detallado.copy()
df_detalle_patrones['dia'] = df_detalle_patrones['dia'].apply(lambda x: lista_dias_full.index(x) + 1).astype(str)
df_detalle_patrones

In [0]:
df_detalle_patrones['nombre_patron'] = df_detalle_patrones['persona'] + '_' + df_detalle_patrones['estacion']
df_detalle_patrones


## Maestro de rotación

In [0]:
df_maestro_patrones = df_detalle_patrones[['nombre_patron', 'estacion']].drop_duplicates()
df_maestro_patrones['detalle'] = df_maestro_patrones['nombre_patron']
df_maestro_patrones

In [0]:
# Completar informacion con la tabla de homologacion
df_maestro_patrones = pd.merge(df_maestro_patrones, df_homologacion, how = 'left', left_on = 'estacion', right_on = 'StationName')
df_maestro_patrones = df_maestro_patrones[['nombre_patron', 'detalle', 'SchedulingGroup', 'SchedulingUnit']]

df_maestro_patrones

## Continuar con el detalle

In [0]:
df_detalle_patrones['esta_trabajando'] = 'Y'
df_detalle_patrones = df_detalle_patrones[['nombre_patron', 'dia', 'esta_trabajando', 'nombre_modelo']]
df_detalle_patrones

In [0]:
patrones = df_detalle_patrones['nombre_patron'].unique()
df_patrones_full = pd.DataFrame({'nombre_patron' : [], 'dia' : []})

for i in range(1, 15):
  _df = pd.DataFrame({'nombre_patron' : patrones})
  _df['dia'] = f"{i}"
  df_patrones_full = pd.concat([df_patrones_full, _df])

df_patrones_full

In [0]:
df_detalle_patrones = pd.merge(df_patrones_full, df_detalle_patrones, how = 'outer', on = ['nombre_patron', 'dia'])
df_detalle_patrones['esta_trabajando'] = df_detalle_patrones['esta_trabajando'].fillna('Y')
df_detalle_patrones['nombre_modelo'] = df_detalle_patrones['nombre_modelo'].fillna('DESCANSO')

df_detalle_patrones

---
# Continuar con la creación del df de modelos
---

## Quitar el "2" en los dias y agrupar teniendo en cuenta: persona, horario (tiempos ini y fin) y dia (ignoramos consultorio)

In [0]:
# Quitar el "2" en los dias
df_modelo_detallado['dia'] = df_modelo_detallado['dia'].str.replace(' 2', '')
df_modelo_detallado

## Agrupar teniendo en cuenta: persona, horario (tiempos ini y fin) y dia

---
\+ todos los cambios nuevos: estacion, cualificacion, turno y nombre_modelo

In [0]:
# Agrupar teniendo en cuenta: persona, horario (tiempos ini y fin)
# y dia (ignoramos consultorio)

# agrupamos porque si tengo la misma persona en el mismo dia (semanas diferentes), solo me deberia quedar una sola vez
# por otro lado, si tengo la misma persona en el mismo dia y tienen horarios diferentes (semanas diferentes) debo dejar los dos

df_modelo_det_agrup = df_modelo_detallado.drop_duplicates(['id_sede', 'persona', 'dia', 'hora_ini', 'hora_fin',
                                                           'cualificacion', 'estacion', 'turno', 'nombre_modelo'])
df_modelo_det_agrup


## "Transponer" los dias

In [0]:
# "Transponer" los dias
df_modelo_det_agrup = df_modelo_det_agrup.pivot_table(index=['id_sede', 'persona', 'hora_ini', 'hora_fin',
                                                             'cualificacion', 'estacion', 'turno', 'nombre_modelo'],
                                                      columns = 'dia', aggfunc = lambda x: 1)
df_modelo_det_agrup = df_modelo_det_agrup.fillna(0)
df_modelo_det_agrup = df_modelo_det_agrup.reset_index()

# agregar las columnas que hagan falta, si no tenemos registros para un dia
# especifico, no existira la columna despues de transponer
campos_faltantes = set(lista_dias) - set(df_modelo_det_agrup.columns)
for campo in campos_faltantes:
  df_modelo_det_agrup[f'{campo}'] = 0.0

df_modelo_det_agrup

---
# Agregar los campos "calculados"





---

## Agrupar sin tener en cuenta las personas

In [0]:
# Agrupar utilizando StationName, ShiftName, QualificationNames y los ini y fin de cada dia

df_modelos_eu = df_modelo_det_agrup.groupby(['hora_ini', 'hora_fin', 'cualificacion', 'estacion',
                                             'turno', 'nombre_modelo'])[lista_dias].agg(['sum'])
df_modelos_eu = df_modelos_eu.reset_index()

# quitar los "sum"
df_modelos_eu.columns = [sublista[0] for sublista in df_modelos_eu.columns]

df_modelos_eu

## Dar formato a la hora

In [0]:
#
df_modelos_eu['hora_ini'] = df_modelos_eu['hora_ini'].astype(str) + ':00'
df_modelos_eu['hora_fin'] = df_modelos_eu['hora_fin'].astype(str) + ':00'

df_modelos_eu

---
# Renombrar y ordenar (parcial)
---

## Modelos

In [0]:
# renombrar
df_modelos_eu = df_modelos_eu.rename(
    columns = {
        'nombre_modelo' : 'ModelName',
        'estacion' : 'StationName',
        'turno' : 'ShiftName',
        'cualificacion' : 'QualificationNames',
    }
)

# ordenar:
nuevo_orden = ['ModelName', 'StationName', 'ShiftName', 'QualificationNames',
               'hora_ini', 'hora_fin'] + lista_dias
df_modelos_eu = df_modelos_eu.reindex(columns = nuevo_orden, fill_value = None)


df_modelos_eu

## Maestro de rotación

In [0]:
# renombrar
df_maestro_patrones = df_maestro_patrones.rename(
    columns = {
        'nombre_patron' : 'RotationPatternName',
        'detalle' : 'Description'
    }
)

df_maestro_patrones

## Detalle de rotación

In [0]:
# renombrar
df_detalle_patrones = df_detalle_patrones.rename(
    columns = {
        'nombre_patron' : 'RotationPatternName',
        'dia' : 'DayNumber',
        'esta_trabajando' : 'IsWorking',
        'nombre_modelo' : 'Model'
    }
)

df_detalle_patrones

---
# Agregar campos restantes ("fijos" y vacios)
---

## Modelos

In [0]:

df_modelos_eu['ModelType'] = 'WORK'
df_modelos_eu['StartDate'] = datetime.now().strftime("%Y-%m-%d")
df_modelos_eu['EndDate'] = '3000-12-31'
df_modelos_eu['DifficultyRating'] = 0
df_modelos_eu['AllowJobOverlap'] = 'N'
df_modelos_eu['CalloutOrder'] = 'NOT_REQUIRED'
df_modelos_eu['CalloutWritesOTEqualization'] = 'NOT_REQUIRED'
df_modelos_eu['FMStatus'] = ''
df_modelos_eu['Description'] = df_modelos_eu['ShiftName']
df_modelos_eu['DbsDataPolicy'] = ''
df_modelos_eu['DbsHierarchyNode'] = ''


for num_dia in range(0, 8):

  df_modelos_eu[f'{lista_dias_wfs[num_dia]}Start'] = df_modelos_eu['hora_ini']
  df_modelos_eu[f'{lista_dias_wfs[num_dia]}End'] = df_modelos_eu['hora_fin']

  df_modelos_eu[f'{lista_dias_wfs[num_dia]}Proficiency'] = ''
  df_modelos_eu[f'{lista_dias_wfs[num_dia]}RequiredHeadCount'] = df_modelos_eu[f'{lista_dias[num_dia]}'].astype(int)

  df_modelos_eu[f'{lista_dias_wfs[num_dia]}OptionalHeadCount'] = ''
  df_modelos_eu[f'{lista_dias_wfs[num_dia]}ScheduleOrder'] = ''
  df_modelos_eu[f'{lista_dias_wfs[num_dia]}LaborStandard'] = ''


for i in range(1, 16):
  df_modelos_eu[f'UserField{i}'] = ''

# ya no se necesitan
df_modelos_eu = df_modelos_eu.drop(['hora_ini', 'hora_fin'] + lista_dias, axis = 1)

df_modelos_eu

## Maestro de rotación

In [0]:
for i in range(1, 11):
  df_maestro_patrones[f'UserField{i}'] = ''

df_maestro_patrones

## Detalle de rotación

In [0]:
df_detalle_patrones['ReportNotation'] = ''
df_detalle_patrones['MinBreakBetweenShifts'] = ''

for i in range(1, 6):
  df_detalle_patrones[f'UserField{i}'] = ''

for i in range(1, 6):
  df_detalle_patrones[f'FMDayUserField{i}'] = ''

df_detalle_patrones



---
# Exportar CSV
---

In [0]:
df_modelos_eu.to_csv(ruta_salidas + 'db_models.csv', index = False)

In [0]:
df_maestro_patrones.to_csv(ruta_salidas + 'rotation_pattern_masters.csv', index = False)

In [0]:
df_detalle_patrones.to_csv(ruta_salidas + 'rotation_pattern_details.csv', index = False)

---
# Validaciones para verificar que todo existe en WFS
---
Si algo no existe se debe crear en WFS antes se subir cualquier archivo.

## Estaciones

In [0]:
min_estaciones = df_modelos_eu['StationName'].dropna().drop_duplicates()
estaciones_faltantes = min_estaciones[ ~min_estaciones.isin(v_estaciones) ]

if len(estaciones_faltantes) > 0:
    estaciones_faltantes.to_excel('WFS_estaciones_faltantes.xlsx', index = False)
    shutil.move('WFS_estaciones_faltantes.xlsx', ruta_salidas)

estaciones_faltantes

## Turnos

In [0]:
min_turnos = df_modelos_eu['ShiftName'].dropna().drop_duplicates()
turnos_faltantes = min_turnos[ ~min_turnos.isin(v_turnos) ]

if len(turnos_faltantes) > 0:
    turnos_faltantes.to_excel('WFS_turnos_faltantes.xlsx', index = False)
    shutil.move('WFS_turnos_faltantes.xlsx', ruta_salidas)

turnos_faltantes

## Cualificaciones

In [0]:
min_cualificaciones = df_modelos_eu[['QualificationNames', 'StationName']].drop_duplicates()

# encontrar el scheduling group para ese station
min_cualificaciones = pd.merge(min_cualificaciones, df_homologacion, how = 'left', on = 'StationName')
min_cualificaciones = min_cualificaciones.drop(['StationName', 'SchedulingGroup'], axis = 1)
min_cualificaciones

In [0]:
cualificaciones_faltantes = min_cualificaciones[ ~min_cualificaciones.isin(v_cualificaciones) ]
cualificaciones_faltantes

In [0]:
min_min_cualificaciones = min_cualificaciones[['QualificationNames']].drop_duplicates()
min_min_cualificaciones['Observaciones'] = 'Existe, pero hay problemas con Scheduling Unit'

cualificaciones_faltantes = pd.merge(cualificaciones_faltantes, min_min_cualificaciones, how = 'left', on = 'QualificationNames')

if len(cualificaciones_faltantes) > 0:
    cualificaciones_faltantes.to_excel('WFS_cualificaciones_faltantes.xlsx', index = False)
    shutil.move('WFS_cualificaciones_faltantes.xlsx', ruta_salidas)

cualificaciones_faltantes

# Enviar a SFTP

In [0]:
# Configuracion de conexion SFTP
# sftp_host = 'FTPLAB03.labsura.local'
sftp_host = '10.51.156.8'
sftp_username = 'FTPCuadrodeTurnos'
sftp_password = 'Ftpcu4dr0d3turn0s'

cnopts = pysftp.CnOpts()
cnopts.hostkeys = None

# conexion SFTP
with pysftp.Connection(host=sftp_host, username=sftp_username, password=sftp_password, cnopts=cnopts) as sftp:
    
    # borrar archivos existentes
    archivos_remotos = sftp.listdir(ruta_salidas_sftp)
    for archivo_remoto in archivos_remotos:
        # borrar
        sftp.remove(ruta_salidas_sftp + archivo_remoto)
        print('Archivo ' + archivo_remoto + ' borrado.')

    # enviar archivos nuevos
    archivos_locales = os.listdir(ruta_salidas)
    for archivo_local in archivos_locales:
        sftp.put(ruta_salidas + archivo_local, ruta_salidas_sftp + archivo_local)
        print('Archivo ' + archivo_local + ' enviado.')




